In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf

In [ ]:
# Télécharger les données de Yahoo Finance
tickers = ["JPM", "AIG", "GS"]
start_date = "2007-01-01"
end_date = "2023-12-31"

# Initialisation du dictionnaire pour stocker les données
data = {}

for ticker in tickers:
    print(f"Téléchargement des données pour {ticker}...")
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    
    # Vérifier si des données sont disponibles
    if not stock_data.empty:
        data[ticker] = stock_data['Close']
    else:
        print(f"Aucune donnée trouvée pour {ticker}.")

# Vérifier si 'data' contient des données valides
if data:
    # Combiner les données en un DataFrame
    adjusted_prices = pd.concat(data.values(), axis=1)
    adjusted_prices.columns = data.keys()  # Nommer les colonnes avec les tickers
    adjusted_prices.index.name = "Date"   # Nommer l'index
    adjusted_prices.reset_index(inplace=True)

    # Sauvegarder les prix ajustés
    adjusted_prices.to_csv("adjusted_prices.csv", index=False)
    print("Données sauvegardées dans 'adjusted_prices.csv'.")
else:
    print("Aucune donnée valide n'a été trouvée pour les tickers spécifiés.")

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd

# Charger la base de données à partir du fichier CSV
adjusted_prices = pd.read_csv("adjusted_prices.csv")

# Afficher les premières lignes de la base de données
print("Aperçu des premières lignes :")
print(adjusted_prices.head())

In [ ]:
# Charger les prix ajustés
adjusted_prices = pd.read_csv("adjusted_prices.csv")
adjusted_prices['Date'] = pd.to_datetime(adjusted_prices['Date'])

# Calculer les rendements logarithmiques
log_returns = adjusted_prices.copy()
for ticker in tickers:
    log_returns[ticker] = np.log(adjusted_prices[ticker] / adjusted_prices[ticker].shift(1))

# Supprimer les valeurs manquantes
log_returns.dropna(inplace=True)

# Sauvegarder les rendements
log_returns.to_csv("log_returns.csv", index=False)

print(log_returns.head())

In [ ]:
import sys
print(sys.executable)

In [ ]:
!{sys.executable} -m pip install arch

In [ ]:
import numpy as np
import pandas as pd
from arch import arch_model
import matplotlib.pyplot as plt
from scipy.stats import kurtosis, skew, jarque_bera
from statsmodels.stats.diagnostic import acorr_ljungbox
from arch import arch_model

# Load log returns
log_returns = pd.read_csv("Log_Returns.csv", parse_dates=["Date"])
log_returns.set_index("Date", inplace=True)

# Extract institutions
JPM = log_returns["JPM"].dropna()
AIG = log_returns["AIG"].dropna()
GS = log_returns["GS"].dropna()

# Calculate skewness and kurtosis
print("Skewness:")
print(f"JPM: {skew(JPM):.6f}")
print(f"AIG: {skew(AIG):.6f}")
print(f"GS: {skew(GS):.6f}")

print("\nKurtosis:")
print(f"JPM: {kurtosis(JPM, fisher=False):.6f}")
print(f"AIG: {kurtosis(AIG, fisher=False):.6f}")
print(f"GS: {kurtosis(GS, fisher=False):.6f}")

# Define a function to estimate GARCH models
def estimate_garch(data, dist):
    return arch_model(data, vol="Garch", p=1, q=1, mean="Zero", dist=dist).fit(disp="off")

# Distributions to test
distributions = ["normal", "t", "ged"]

# Function to collect model metrics
def model_metrics(model):
    residuals = model.resid / model.conditional_volatility
    return {
        "Omega": model.params["omega"],
        "Alpha1": model.params["alpha[1]"],
        "Beta1": model.params["beta[1]"],
        "Shape": model.params.get("nu", model.params.get("lambda", None)),
        "AIC": model.aic,
        "BIC": model.bic,
        "LogLik": model.loglikelihood,
        "LjungBox_p": acorr_ljungbox(residuals, lags=[10], return_df=True)["lb_pvalue"].iloc[0]
    }

# Diagnostic checks
def diagnostic_checks(model):
    residuals = model.resid / model.conditional_volatility
    return {
        "LjungBox_Std": acorr_ljungbox(residuals, lags=[10], return_df=True)["lb_pvalue"].iloc[0],
        "LjungBox_Sq": acorr_ljungbox(residuals**2, lags=[10], return_df=True)["lb_pvalue"].iloc[0],
        "JB_p": jarque_bera(residuals)[1],
        "Skewness": skew(residuals),
        "Kurtosis": kurtosis(residuals, fisher=False)
    }

# Estimation for each institution
results = {}
for ticker, data in {"JPM": JPM, "AIG": AIG, "GS": GS}.items():
    print(f"\nEstimating GARCH models for {ticker}...")
    volatility_est = {dist: estimate_garch(data, dist) for dist in distributions}
    metrics = {dist: model_metrics(model) for dist, model in volatility_est.items()}
    diagnostics = {dist: diagnostic_checks(model) for dist, model in volatility_est.items()}
    results[ticker] = {
        "metrics": pd.DataFrame(metrics).T,
        "diagnostics": pd.DataFrame(diagnostics).T
    }

# Print results
for ticker, result in results.items():
    print(f"\nMetrics for {ticker}:")
    print(result["metrics"].round(6))
    print(f"\nDiagnostics for {ticker}:")
    print(result["diagnostics"].round(6))

# Plotting
def plot_diagnostics(ticker, models):
    dist_full_names = {
        "normal": "Normal Distribution",
        "t": "Student-t Distribution",
        "ged": "Generalized Error Distribution"
    }
    plt.figure(figsize=(12, 8))
    for dist, model in models.items():
        residuals = model.resid / model.conditional_volatility
        plt.subplot(3, 3, list(models.keys()).index(dist) * 3 + 1)
        plt.title(f"{ticker} - Q-Q Plot ({dist_full_names[dist]})")
        qq = np.sort(residuals)
        plt.plot(qq, np.sort(np.random.normal(0, 1, len(qq))), 'o', alpha=0.5)
        plt.plot([-3, 3], [-3, 3], 'r--')
        plt.subplot(3, 3, list(models.keys()).index(dist) * 3 + 2)
        plt.title(f"{ticker} - ACF ({dist_full_names[dist]})")
        plt.acorr(residuals, maxlags=10)
        plt.subplot(3, 3, list(models.keys()).index(dist) * 3 + 3)
        plt.title(f"{ticker} - ACF of Squared Residuals ({dist_full_names[dist]})")
        plt.acorr(residuals**2, maxlags=10)
    plt.tight_layout()
    plt.show()

for ticker, data in {"JPM": JPM, "AIG": AIG, "GS": GS}.items():
    plot_diagnostics(ticker, {dist: estimate_garch(data, dist) for dist in distributions})

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from scipy.stats import t, norm

# 1. Load the data
log_returns = pd.read_csv("Log_Returns.csv", index_col="Date", parse_dates=True)

# 2. Parameters
alpha = 0.05  # 5% risk level
rolling_windows = [250, 500]  # 250-day and 500-day rolling window
institutions = ["JPM", "AIG", "GS"]

# 3. Define the best models for each institution
models = {
    "JPM": "Garch",   # GARCH(1,1) with Student-t distribution
    "AIG": "EGarch",  # EGARCH(1,1) with Skew Student-t distribution
    "GS": "Garch"     # GARCH(1,1) with Student-t distribution
}

# 4. Define the best distribution for each institution
distributions = {
    "JPM": "studentst",
    "AIG": "ged",  # Skew Student-t is not directly supported in arch package
    "GS": "studentst"
}

# 5. Store results
results = {}
figure_number = 1  # Initialize figure numbering

# 6. Loop over institutions and rolling windows
for inst in institutions:
    print(f"Processing {inst}...")

    for rolling_window in rolling_windows:
        returns = log_returns[inst].dropna()
        VaR_HS = []
        VaR_GARCH = []
        forecast_dates = []

        # Rolling window estimation
        for i in range(rolling_window, len(returns)):
            # Define rolling data
            rolling_data = returns.iloc[i - rolling_window:i]

            # Store forecast date
            forecast_dates.append(returns.index[i])

            # Historical Simulation VaR
            VaR_HS.append(np.percentile(rolling_data, alpha * 100))

            # Fit GARCH model
            spec = arch_model(
                rolling_data,
                vol=models[inst],
                p=1,
                q=1,
                dist=distributions[inst],
                mean="Zero"
            )
            model = spec.fit(disp="off")

            # Forecast conditional volatility
            sigma_forecast = model.conditional_volatility[-1]

            # GARCH VaR
            VaR_GARCH.append(
                sigma_forecast * t.ppf(alpha, df=5) if distributions[inst] == "studentst" 
                else sigma_forecast * norm.ppf(alpha)
            )

        # Store results
        results[(inst, rolling_window)] = pd.DataFrame({
            "Date": forecast_dates,
            "VaR_HS": VaR_HS,
            "VaR_GARCH": VaR_GARCH
        }).set_index("Date")

        # Plot results
        plt.figure(figsize=(12, 6))
        plt.plot(results[(inst, rolling_window)].index, results[(inst, rolling_window)]["VaR_HS"], 
                 label=f"VaR HS ({rolling_window}-day)", linestyle="--", color="blue")
        plt.plot(results[(inst, rolling_window)].index, results[(inst, rolling_window)]["VaR_GARCH"], 
                 label=f"VaR GARCH ({rolling_window}-day)", linestyle="--", color="red")
        plt.plot(log_returns.loc[results[(inst, rolling_window)].index, inst], 
                 label="Returns", color="black", alpha=0.5)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Values", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()

        # Add title as figure caption at the bottom
        title = f"Fig. {figure_number} — VaR Forecasts for {inst} using a {rolling_window}-day Rolling Window"
        plt.figtext(0.5, -0.05, title, wrap=True, horizontalalignment='center', fontsize=10)
        plt.show()

        # Increment the figure number
        figure_number += 1

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from arch import arch_model
from scipy.stats import t, norm

# 1. Load the data
log_returns = pd.read_csv("Log_Returns.csv", index_col="Date", parse_dates=True)

# 2. Parameters
alpha = 0.05  # 5% risk level
rolling_windows = [250, 500]  # 250-day and 500-day rolling window
institutions = ["JPM", "AIG", "GS"]

# 3. Define the best models for each institution
models = {
    "JPM": "Garch",   # GARCH(1,1) with Student-t distribution
    "AIG": "EGarch",  # EGARCH(1,1) with Skew Student-t distribution
    "GS": "Garch"     # GARCH(1,1) with Student-t distribution
}

# 4. Define the best distribution for each institution
distributions = {
    "JPM": "studentst",
    "AIG": "ged",  # Skew Student-t is not directly supported in arch package
    "GS": "studentst"
}

# 5. Store results
results = {}
figure_number = 7  # Start numbering figures from 7

# 6. Loop over institutions and rolling windows
for inst in institutions:
    print(f"Processing {inst}...")

    for rolling_window in rolling_windows:
        returns = log_returns[inst].dropna()
        ES_HS = []
        ES_GARCH = []
        forecast_dates = []

        # Rolling window estimation
        for i in range(rolling_window, len(returns)):
            # Define rolling data
            rolling_data = returns.iloc[i - rolling_window:i]

            # Store forecast date
            forecast_dates.append(returns.index[i])

            # Historical Simulation ES
            below_threshold = rolling_data[rolling_data <= np.percentile(rolling_data, alpha * 100)]
            ES_HS.append(np.mean(below_threshold) if len(below_threshold) > 0 else None)

            # Fit GARCH model
            spec = arch_model(
                rolling_data,
                vol=models[inst],
                p=1,
                q=1,
                dist=distributions[inst],
                mean="Zero"
            )
            model = spec.fit(disp="off")

            # Forecast conditional volatility
            sigma_forecast = model.conditional_volatility[-1]

            # GARCH ES Calculation
            if distributions[inst] == "studentst":
                ES_GARCH.append(-sigma_forecast * (t.pdf(t.ppf(alpha, df=5), df=5) / alpha))
            else:
                ES_GARCH.append(-sigma_forecast * (norm.pdf(norm.ppf(alpha)) / alpha))

        # Store results
        results[(inst, rolling_window)] = pd.DataFrame({
            "Date": forecast_dates,
            "ES_HS": ES_HS,
            "ES_GARCH": ES_GARCH
        }).set_index("Date")

        # Plot results
        data = results[(inst, rolling_window)]  # Correctly retrieve the stored data
        plt.figure(figsize=(12, 6))
        plt.plot(data.index, data["ES_HS"], label=f"ES HS ({rolling_window}-day)", linestyle="--", color="blue")
        plt.plot(data.index, data["ES_GARCH"], label=f"ES GARCH ({rolling_window}-day)", linestyle="--", color="red")
        plt.plot(log_returns.loc[data.index, inst], label="Returns", color="black", alpha=0.5)
        plt.xlabel("Date", fontsize=12)
        plt.ylabel("Returns / ES", fontsize=12)
        plt.legend(fontsize=10)
        plt.tight_layout()

        # Add title as figure caption at the bottom
        title = f"Fig. {figure_number} — Forecast for ES using a {rolling_window}-day Rolling Window for {inst}"
        plt.figtext(0.5, -0.05, title, wrap=True, horizontalalignment='center', fontsize=10)
        plt.show()

        # Increment the figure number
        figure_number += 1